# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 

### Associated lectures: [Lectures 17 and 18](https://github.com/UBC-CS/cpsc330-2024s/tree/main/lectures)

**See PrairieLearn for _due date_ and _submission_**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size $\leq$ 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024s/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [5]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [6]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [7]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [8]:
glove_wiki_vectors.most_similar("vegetable")

[('vegetables', 0.7828993201255798),
 ('edible', 0.7044364809989929),
 ('chicken', 0.6951989531517029),
 ('fruit', 0.6914322972297668),
 ('corn', 0.680595338344574),
 ('tomato', 0.6755445003509521),
 ('oils', 0.665894627571106),
 ('potato', 0.6595883369445801),
 ('beans', 0.6558663845062256),
 ('wheat', 0.6548263430595398)]

In [9]:
glove_wiki_vectors.most_similar("monkey")

[('cat', 0.7288709878921509),
 ('monkeys', 0.7264204621315002),
 ('mouse', 0.6769240498542786),
 ('ape', 0.6653543710708618),
 ('squirrel', 0.6539695262908936),
 ('spider', 0.6474379301071167),
 ('elephant', 0.6434566974639893),
 ('rat', 0.6417514681816101),
 ('snake', 0.6381354928016663),
 ('pig', 0.6356362104415894)]

In [10]:
glove_wiki_vectors.most_similar("computer")

[('computers', 0.875198245048523),
 ('software', 0.8373122215270996),
 ('technology', 0.7642159461975098),
 ('pc', 0.7366448640823364),
 ('hardware', 0.7290390729904175),
 ('internet', 0.7286775708198547),
 ('desktop', 0.7234441637992859),
 ('electronic', 0.7221828699111938),
 ('systems', 0.7197922468185425),
 ('computing', 0.7141730189323425)]

In [11]:
glove_wiki_vectors.most_similar("pizza")

[('sandwich', 0.7488481402397156),
 ('sandwiches', 0.7221846580505371),
 ('bread', 0.7006676197052002),
 ('pizzas', 0.7004501223564148),
 ('burger', 0.6977595686912537),
 ('snack', 0.6840099692344666),
 ('taco', 0.678828239440918),
 ('pie', 0.677691638469696),
 ('chicken', 0.6714670658111572),
 ('burgers', 0.6690315008163452)]

In [12]:
glove_wiki_vectors.most_similar("jambalaya")

[('gumbo', 0.7292833924293518),
 ('crawfish', 0.5627177953720093),
 ('dixieland', 0.5162704586982727),
 ('etouffee', 0.5089052319526672),
 ('schmaltz', 0.5027944445610046),
 ('cajun', 0.5027632117271423),
 ('paella', 0.4975961744785309),
 ('andouille', 0.4960995018482208),
 ('pozole', 0.4913918972015381),
 ('gravy', 0.4907839000225067)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [13]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]
word_pairs[0]

('coast', 'shore')

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [14]:

coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")

clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")



<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [17]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [18]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Type your answer here, replacing this text._

In [19]:
print(glove_wiki_vectors.similarity("man", "smart"))
print(glove_wiki_vectors.similarity("woman", "smart"))

0.47782284
0.36801583


In [20]:
print(glove_wiki_vectors.similarity("man", "stupid"))
print(glove_wiki_vectors.similarity("woman", "stupid"))

0.40549734
0.29395166


In [21]:
print(glove_wiki_vectors.similarity("man", "lazy"))
print(glove_wiki_vectors.similarity("woman", "lazy"))

0.32689095
0.28464356


In [22]:
print(glove_wiki_vectors.similarity("apple", "delicious"))
print(glove_wiki_vectors.similarity("orange", "delicious"))

0.45273808
0.3394756


<!-- END QUESTION -->
We see that the embeddings have 'man' more similar to both 'smart' and 'stupid' than 'woman' by a significant margin. There appears to be biases in both directions here. I think word frequency may impact similarity causing biases during training. We see that apple is much more similar to delicious than orange, while that is completely subjective, the word apple may simply be used more often. 
<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. I would need to check more examples to have a stronger judgement on the quality of the embeddings, but biases appear to be present. Perhaps there is a way to control for word frequency in the training process.
2. If a model influences important decisions like hiring, then bias could negatively impact certain demographic groups. 

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [23]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [24]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [25]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [26]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [34]:
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

from sklearn.metrics import classification_report

pipeline = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression()
)

pipeline.named_steps["countvectorizer"].fit(X_train)
X_train_vec = pipeline.named_steps["countvectorizer"].transform(X_train)
pipeline.fit(X_train, y_train)

bow_train_accuracy = pipeline.score(X_train, y_train)
bow_test_accuracy = pipeline.score(X_test, y_test)
bow_report = classification_report(y_test, pipeline.predict(X_test), digits=2) # use digits=2

print("Train accuracy {:.2f}".format(bow_train_accuracy))
print("Test accuracy {:.2f}".format(bow_test_accuracy))

print(bow_report)

Train accuracy 0.96
Test accuracy 0.82
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.55      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.72       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.82      4238



In [36]:
X_train_embeddings = None
X_test_embeddings = None

X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [40]:
emb_train_accuracy = None
emb_test_accuracy = None
emb_report = None # use digits=2

reg = LogisticRegression(max_iter = 2000)
reg.fit(X_train_embeddings, y_train)

emb_train_accuracy = reg.score(X_train_embeddings, y_train)
emb_test_accuracy = reg.score(X_test_embeddings, y_test)
print("Train accuracy {:.2f}".format(emb_train_accuracy))
print("Test accuracy {:.2f}".format(emb_test_accuracy))
emb_report = classification_report(y_test, reg.predict(X_test_embeddings), digits=2)
print(emb_report)

Train accuracy 0.85
Test accuracy 0.79
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise       0.67      0.73      0.70        74
         leisure       0.71      0.65      0.68       407
          nature       0.66      0.72      0.69        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer uses counts of tokens, while average-embedding uses pre-trained vector representations of words
2. I think that logistic regression with bag of words would be the most interpretable, since we can directly observe coefficients which indicate the direct contribution of each token to the prediction. 
3. Yes using pre-trained embeddings is transfer learning, but it isn't an improvement on bag of words. Perhaps embeddings trained on a different source would improve accuracy.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [41]:
from sklearn.datasets import fetch_20newsgroups

In [42]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [43]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [44]:
nlp = en_core_web_md.load(disable=["parser", "ner"])


In [85]:
import re

def removeChars(text):
    text = re.sub("""[\/\*\n\r]""", "", text)
    return text

def preproc(document,
           irrelevant_pos_tags = [
            'PUNCT',   # Punctuation
            'DET',     # Determiner
            'CCONJ',   # Coordinating conjunction
            'SCONJ',   # Subordinating conjunction
            'ADP',     # Adposition
            'PART',    # Particle
            'INTJ'     # Interjection
            ]):
    result = []
    for t in document:
        if t.like_url:
            result.append("url")
        elif t.like_email:
            result.append("email")
        elif t.like_num:  
            result.append("num")    
        elif (
            t.is_stop == False
            and len(t) >= 3
            and t.pos_ not in irrelevant_pos_tags
        ):
            result.append(t.lemma_.lower())
    return " ".join(result)

In [86]:
df["text_pp"] = df["text"].apply(removeChars)


In [87]:
df["text_pp"] = [
    preproc(text) for text in nlp.pipe(df["text"].tolist())
] 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

I removed the following characters: "*", "~", "/", and newlines, carriage returns, and numbers. 

Then I replaced urls and emails by general strings for each as these likely won't contribute to topic modelling. Stop words were removed because they are so common they won't help determine topic. 

Then I removed stopwords, parts-of-speech tags (including punctuation) commonly considered irrelevent for topic modelling, and tokens with fewer than 3 occurences. Finally I only included sentence segmented lower case lemmatized tokens. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Type your answer here, replacing this text._

In [88]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df["text_pp"])
X_vectorized = vectorizer.transform(df["text_pp"])

nm = 4

topmod = LatentDirichletAllocation(n_components = nm, learning_method = "batch", max_iter = 10)

topics = topmod.fit_transform(X_vectorized)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [89]:
sort = np.argsort(topmod.components_, axis = 1)[:, ::-1]
names = np.array(vectorizer.get_feature_names_out())

In [90]:
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names = names,
    sorting = sort,
    topics_per_chunk = 5,
    n_words = 20
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           num           
god           people        url           file          
think         say           email         use           
believe       go            game          gun           
jesus         know          team          know          
know          come          play          window        
people        armenian      year          people        
good          think         run           url           
entry         time          player        right         
say           armenians     win           image         
time          turkish       season        israel        
church        tell          new           state         
file          kill          good          problem       
bible         year          include       law           
thing         want          available     system        
way           see           lea

<!-- END QUESTION -->
Topic 0 is clearly religion.
Topic 1 looks related to geopolitics and conflict. 
Topic 2 looks sports related.
Topic 3 includes some computer terms and some I would put in a political topic. With more time I would try to improve this model.
<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [93]:
for i in range(5):
    print("\nDOC", df.iloc[i]["text"])
    print("\nTopic assignment: ", topics[i])
    print("_________________")


DOC You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?




Topic assignment:  [0.00423818 0.00

<!-- END QUESTION -->
2.

The first document is some type of legal commentary, and it is strongly assigned to topic 3, which may be political. This makes sense. 

The second is a communication about an algorithm, and it was assigned to topic 2 which appears to be sports related, which is confusing, maybe sports and computer science share some vocabulary. 

The third is some sort of communication about graphics in news media, which was assigned to the computer/political topic which makes sense. 

The fourth is a question about code which was equally assigned to topics 1 and 3, indicating my model may not be optimal. 

The fifth is a commentary on the Armenian Genocide, and it was assigned to topic 1 which appears to include words on geoplitics and conflict. 
<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. 
    - Decision tree: this already works for multiclass
      
    - KNNs: this already works for multiclass
    - Random Forest: this already works for multiclass
    - Logistic regression: We are using a one vs rest approach
    - SVM RBF:  by default, sklearn uses a one vs one approach
2. Transfer learning is using pretrained models and tuning them for our purposes, since completely training such a model is expensive.
3. This allows the persistence of pretrained layers and we can only modify the final layer with our tuning.
   
   


<!-- END QUESTION -->

<br><br><br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using PrairieLearn.
4. Make sure that the plots and output are rendered properly in your submitted file.